# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language(could be sequences, text or speech).

Another common term for NLP problems is sequence problems(seq2seq).

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fdc50f8d-486a-8b37-ba30-bcd98e29004d)


## Get Helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-06-08 19:37:38--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-08 19:37:38 (89.5 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get a Text Dataset

The dataset we're goin to using is Kaggle's introduction to NLP dataset (test sample of Tweets labelled as disaster or not disaster).

See the original source here: 
https://www.kaggle.com/competitions/nlp-getting-started/overview

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-08 19:37:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 173.194.74.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-06-08 19:37:44 (121 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission_df = pd.read_csv("sample_submission.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
sample_submission_df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher then total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Blood Group  A +ve is associated with Gastric Carcinoma says text book...Another fragile gene in my body....

---

Target: 0 (not real disaster)
Text:
I liked a @YouTube video http://t.co/45TWHJ0l6m RomanAtwoodVlogs | RESCUED SICK KITTENS!!

---

Target: 1 (real disaster)
Text:
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_

---

Target: 0 (not real disaster)
Text:
@YoungHeroesID 4. Lava Blast Power Red #PantherAttack

---

Target: 1 (real disaster)
Text:
Central Mass. fruit trees escape heavy damage after wind hail http://t.co/VbFfodtP6M

---



### Split data into training and validation dataset

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # using 10% for validation
                                                                            random_state=42)

In [13]:
# Check the lenghths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned).


### Text Vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabulary (automatically add <OOV>)? 
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create group of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do want your sequences to be?
                                    pad_to_max_tokens=False
                                    )

In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it
sample_sentence = "there's a flood in my house!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 318,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose random sentences from the training datasets and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectoized version:")
text_vectorizer([random_sentence])

Original text:
 Philadelphia EaglesÛª Jordan Matthews Is Going To Explode In 2015 http://t.co/rRq1ildkiL #news #hotnewscake        

Vectoized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[4923,    1, 2916,    1,    9,  104,    5,  332,    4,  208,    1,
          58,    1,    0,    0]])>

In [22]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique qords in out training sentences
top_5_words = words_in_vocab[:5] # get the most comman words
bottom_5_words = words_in_vocab[-5:] # get the least comman words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comman words: {top_5_words}")
print(f"5 least comman words: {bottom_5_words}")

Number of words in vocab: 10000
5 most comman words: ['', '[UNK]', 'the', 'a', 'in']
5 least comman words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer

To make our embedding, we are goin to use tensorflow embedding layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean eaach token gets represented by a vector 100 long
* `input_length` = lenght of the sequences being passes to the embedding layer

In [23]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # output_shape
                            embeddings_initializer="uniform",
                            input_length=max_length # how long is each input
                            )

embedding

In [24]:
# get a random sentence from the training set
random_sentences = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Philadelphia EaglesÛª Jordan Matthews Is Going To Explode In 2015 http://t.co/rRq1ildkiL #news #hotnewscake        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04655193,  0.04288021, -0.0203482 , ...,  0.03735841,
         -0.03050013, -0.036662  ],
        [-0.01717655,  0.03999405,  0.00926604, ...,  0.03759272,
         -0.0137833 , -0.02846684],
        [-0.0432953 , -0.01209273, -0.04075279, ..., -0.04391695,
          0.03526929, -0.0406773 ],
        ...,
        [-0.01717655,  0.03999405,  0.00926604, ...,  0.03759272,
         -0.0137833 , -0.02846684],
        [-0.00860078,  0.02972106,  0.04614032, ...,  0.00613161,
          0.02967565, -0.02650087],
        [-0.00860078,  0.02972106,  0.04614032, ...,  0.00613161,
          0.02967565, -0.02650087]]], dtype=float32)>

In [25]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-4.65519316e-02,  4.28802110e-02, -2.03482043e-02,  4.99215238e-02,
         4.26987074e-02, -3.28352079e-02, -4.09862399e-02,  2.70840637e-02,
        -3.92967835e-02,  1.43589415e-02, -3.44152339e-02,  3.65481414e-02,
        -4.09152396e-02, -1.30370259e-02, -2.67330296e-02,  3.19952704e-02,
         9.50766727e-03,  3.24375294e-02, -4.55432311e-02,  1.87557675e-02,
        -8.13783333e-03,  9.59850475e-03, -3.96970510e-02,  4.72205020e-02,
        -4.52511013e-05,  3.23466994e-02,  1.51022933e-02,  1.81731097e-02,
        -8.61109421e-03,  3.10622342e-02,  1.82059444e-02, -3.44387069e-02,
         4.15584780e-02,  4.42743935e-02, -1.39148347e-02, -3.95584926e-02,
         1.24441497e-02,  1.93212740e-02, -4.99494188e-02,  4.84890677e-02,
        -1.58220045e-02, -1.82842724e-02, -2.22937465e-02,  2.29627974e-02,
         9.18854401e-03, -1.49598941e-02, -2.78004054e-02, -4.46413867e-02,
         1.60212778e-02,  3.20310108e-0

## Modeling a text dataset (running a series of experiments)

Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0 : Naive Bayes (baseline), this is from Sklearn ML map:
* Model 1: Feed-Forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 iwth 10% of training data

How are we going to approch all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's import to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers...

> **Note:** it's comman practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # converts words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# fit the pipeline to the traininf data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
baseline_score

0.7926509186351706

In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, howver, this will be easily fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following matircs:
* Accuracy
* Precision
* Recall
* F1-score


In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [32]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of numberized input
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [33]:
model_1.summary();

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [34]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230608-193752
Epoch 1/5
215/215 [==============================] - 28s 113ms/step - loss: 0.6134 - accuracy: 0.6913 - val_loss: 0.5364 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4426 - accuracy: 0.8200 - val_loss: 0.4674 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3471 - accuracy: 0.8608 - val_loss: 0.4556 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2846 - accuracy: 0.8899 - val_loss: 0.4652 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2379 - accuracy: 0.9117 - val_loss: 0.4791 - val_accuracy: 0.7887


In [36]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4791 - accuracy: 0.7887


[0.4790566861629486, 0.7887139320373535]

In [37]:
#  Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [38]:
# Look at the single prediction
model_1_pred_probs[0]

array([0.35921073], dtype=float32)

In [39]:
# Look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.35921073],
       [0.80579793],
       [0.99787486],
       [0.09064739],
       [0.12582473],
       [0.93283373],
       [0.9215917 ],
       [0.9936767 ],
       [0.96944237],
       [0.21585235]], dtype=float32)

In [40]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [41]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)

In [43]:
model_1_results

{'accuracy': 78.87139107611549,
 'precision': 0.7958623979922417,
 'recall': 0.7887139107611548,
 'f1': 0.7850561936710079}

In [44]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [45]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned Embedding

In [46]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [47]:
# Model 1 Summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [48]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned from 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output dimensions of embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handly tool called projector: http://projector.tensorflow.org/

And TensorFlow also has an incredible guid on word embedding.

In [49]:
# Create embedding files (we got this from TensorFlow's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Model 2: LSTM

LSTM = long short term memory (one of  the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)


In [51]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [52]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [53]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [54]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230608-193837
Epoch 1/5
215/215 [==============================] - 15s 50ms/step - loss: 0.2279 - accuracy: 0.9178 - val_loss: 0.5236 - val_accuracy: 0.7913
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1569 - accuracy: 0.9431 - val_loss: 0.6790 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1322 - accuracy: 0.9514 - val_loss: 0.6563 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1109 - accuracy: 0.9585 - val_loss: 0.7057 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0874 - accuracy: 0.9667 - val_loss: 0.8442 - val_accuracy: 0.7730


In [55]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.02306326],
       [0.75627977],
       [0.99956244],
       [0.11865076],
       [0.00100575],
       [0.9979736 ],
       [0.9118755 ],
       [0.999673  ],
       [0.99954814],
       [0.65475076]], dtype=float32)

In [56]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [57]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.29658792650919,
 'precision': 0.7734793297631878,
 'recall': 0.7729658792650919,
 'f1': 0.7715589628566201}

In [58]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [59]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each otherm you need return_sequences as True.
# x = layers.LSTM(64)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [60]:
# Get summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [61]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [62]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230608-193921
Epoch 1/5
215/215 [==============================] - 15s 58ms/step - loss: 0.1599 - accuracy: 0.9356 - val_loss: 0.7643 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0837 - accuracy: 0.9698 - val_loss: 0.8078 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0702 - accuracy: 0.9733 - val_loss: 0.8536 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0600 - accuracy: 0.9765 - val_loss: 1.2058 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0546 - accuracy: 0.9764 - val_loss: 1.2187 - val_accuracy: 0.7730


In [63]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 1s 4ms/step


array([[5.84317895e-04],
       [7.04981804e-01],
       [9.99845743e-01],
       [1.35580704e-01],
       [5.76184539e-05],
       [9.99496341e-01],
       [9.18376982e-01],
       [9.99909759e-01],
       [9.99861121e-01],
       [8.41011584e-01],
       [1.96910521e-04],
       [9.89496171e-01],
       [1.27418156e-04],
       [2.03263775e-01],
       [6.36534751e-05],
       [8.53366044e-04],
       [1.77199719e-04],
       [1.37405194e-04],
       [6.58142492e-02],
       [9.99441683e-01],
       [9.99928236e-01],
       [3.60036065e-05],
       [9.99762475e-01],
       [5.07467485e-04],
       [9.99911904e-01],
       [9.99885082e-01],
       [3.92167974e-04],
       [2.32894352e-04],
       [1.48016756e-04],
       [3.08365732e-01],
       [9.95274067e-01],
       [2.87012750e-04],
       [3.52956937e-03],
       [8.48330907e-04],
       [2.41176754e-01],
       [4.31570619e-01],
       [9.99642968e-01],
       [3.71968776e-01],
       [1.83439162e-02],
       [9.99886513e-01],


In [64]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [65]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'precision': 0.7745161732477092,
 'recall': 0.7729658792650919,
 'f1': 0.7709704727215051}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, bidirectional RNN goes left to right as well as right to left.

In [66]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [67]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [68]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [69]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230608-194005
Epoch 1/5
215/215 [==============================] - 17s 59ms/step - loss: 0.1083 - accuracy: 0.9675 - val_loss: 0.9017 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0544 - accuracy: 0.9771 - val_loss: 0.9953 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0476 - accuracy: 0.9780 - val_loss: 1.1629 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0410 - accuracy: 0.9796 - val_loss: 1.2477 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0436 - accuracy: 0.9804 - val_loss: 1.3363 - val_accuracy: 0.7703


In [70]:
# Make predictions with our bidirectinal model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[3.3034192e-04],
       [7.2661716e-01],
       [9.9969053e-01],
       [2.8957081e-01],
       [1.7404622e-05],
       [9.9923384e-01],
       [2.8131768e-01],
       [9.9995220e-01],
       [9.9984789e-01],
       [9.9768949e-01]], dtype=float32)

In [71]:
# Convert pred probs to pred lables
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [72]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.03412073490814,
 'precision': 0.7709806133638261,
 'recall': 0.7703412073490814,
 'f1': 0.7688058109983439}

In [73]:
model_3_results

{'accuracy': 77.29658792650919,
 'precision': 0.7745161732477092,
 'recall': 0.7729658792650919,
 'f1': 0.7709704727215051}

In [74]:
model_2_results

{'accuracy': 77.29658792650919,
 'precision': 0.7734793297631878,
 'recall': 0.7729658792650919,
 'f1': 0.7715589628566201}

In [75]:
model_1_results

{'accuracy': 78.87139107611549,
 'precision': 0.7958623979922417,
 'recall': 0.7887139107611548,
 'f1': 0.7850561936710079}

## Convolutional Neural Networks for Text (and other types of sequences)

We've used CNNs for Images but images are typically 2D (height x width)... however, our text data is 1D.

Previously we've Conv2D for our image data but now Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)


### Model 5: Conv1D

In [76]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="same") # default = "valid", the output is smaller than input.
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to 'get the most important feature' or 'get the feature wuth the highest value'.

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [77]:
# embedding_test

In [78]:
# conv_1d_output

In [79]:
# max_pool_output

In [80]:
# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our Conv1D model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [81]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230608-194051
Epoch 1/5
215/215 [==============================] - 13s 48ms/step - loss: 0.1256 - accuracy: 0.9623 - val_loss: 0.8756 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0749 - accuracy: 0.9714 - val_loss: 1.0441 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0606 - accuracy: 0.9762 - val_loss: 1.1236 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0541 - accuracy: 0.9783 - val_loss: 1.1998 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0497 - accuracy: 0.9778 - val_loss: 1.2526 - val_accuracy: 0.7546


In [82]:
# Make some predictions with our Conv1D model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.3109030e-01],
       [9.7646070e-01],
       [9.9995816e-01],
       [1.0030780e-01],
       [1.7877454e-08],
       [9.9742246e-01],
       [9.9076080e-01],
       [9.9998283e-01],
       [9.9999976e-01],
       [9.5580453e-01]], dtype=float32)

In [83]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [84]:
# Evaluate model 5 predictions
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 75.45931758530183,
 'precision': 0.7542875283780146,
 'recall': 0.7545931758530183,
 'f1': 0.7536177450076731}

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we've build a few of our models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder.

In [85]:
sample_sentence

"there's a flood in my house!"

In [86]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.02340305  0.03493701  0.02300119 -0.02392508  0.04161692  0.07572431
  0.06496856  0.04330083 -0.00833357  0.0004686   0.02908624 -0.02418867
 -0.00637887  0.08317785  0.06109193 -0.04400333 -0.00283728 -0.05337582
  0.00288018 -0.05230249 -0.0235239   0.04527824  0.05523086  0.01861511
  0.01275493 -0.05692335 -0.0126655  -0.0373417  -0.00399699 -0.06815418
 -0.05138086  0.03389442 -0.04909004 -0.00490341  0.00591681 -0.07060669
  0.04888052  0.02775292 -0.00262368 -0.05406917  0.02959206 -0.00803853
  0.00200852  0.04273778 -0.10710076 -0.03185551 -0.00808867  0.02025404
 -0.06277059  0.02845634], shape=(50,), dtype=float32)


In [87]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.02340305,  0.03493701,  0.02300119, ..., -0.03413377,
         0.05832232, -0.02021496],
       [ 0.03485871, -0.08845595, -0.01677879, ..., -0.02750706,
         0.03230236, -0.00820089]], dtype=float32)>

In [88]:
# Create keras layer using the USE pretrained layer from tenosrflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype="string",
                                        trainable=False,
                                        name="USE")

In [89]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_6_USE")

# Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 256,880,129
Trainable params: 82,305
Non-trainable params: 256,797,824
_________________________________________________________________


In [90]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230608-194135
Epoch 1/5
215/215 [==============================] - 8s 20ms/step - loss: 0.4681 - accuracy: 0.7873 - val_loss: 0.4316 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3923 - accuracy: 0.8282 - val_loss: 0.4226 - val_accuracy: 0.8163
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3622 - accuracy: 0.8464 - val_loss: 0.4308 - val_accuracy: 0.8268
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3306 - accuracy: 0.8629 - val_loss: 0.4345 - val_accuracy: 0.8045
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2846 - accuracy: 0.8875 - val_loss: 0.4461 - val_accuracy: 0.8071


In [91]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.05778324],
       [0.8003798 ],
       [0.99975127],
       [0.12937564],
       [0.6975037 ],
       [0.9509579 ],
       [0.99632853],
       [0.9919058 ],
       [0.9952922 ],
       [0.14023574]], dtype=float32)

In [92]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [93]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 80.70866141732283,
 'precision': 0.8094980410500522,
 'recall': 0.8070866141732284,
 'f1': 0.805391095318273}

In [94]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7:  TF Hub Pretrained USE with 10% of training data.

Transfer learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data.

In [95]:
## Note: Making data splits like below leads to data leakage (model_7 trained on 10% data, outperforms model_6 trained on 100% data.)

# # Create subsets of 10% of the training data
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
# # train_10_percent.head(), len(train_10_percent)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()
# len(train_sentences_10_percent), len(train_labels_10_percent)

> **Note:** Be *very* careful when creating training/val/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of data outperforming the same model trained on 100% of data) trust you gut and go back through find where the error may lie.

In [96]:
# Making a better dataset split (no data leakage)
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [97]:
# Check the number of each label in the updated training data subset
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [99]:
# Check the number of targets in our subset of data
# train_10_percent["target"].value_counts()

A model the same as a previous model you've created you can use the `tf.keras.models.clone_model()` method.

In [100]:
# Let's build a model the same as model_6
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_7_USE")

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary
model_7.summary()


Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 256,880,129
Trainable params: 82,305
Non-trainable params: 256,797,824
_________________________________________________________________


In [101]:
# Fit the model to the 10% training data subsets
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20230608-194235
Epoch 1/5
22/22 [==============================] - 3s 47ms/step - loss: 0.6593 - accuracy: 0.5956 - val_loss: 0.6329 - val_accuracy: 0.5827
Epoch 2/5
22/22 [==============================] - 1s 28ms/step - loss: 0.5248 - accuracy: 0.7839 - val_loss: 0.5154 - val_accuracy: 0.7861
Epoch 3/5
22/22 [==============================] - 1s 27ms/step - loss: 0.3979 - accuracy: 0.8307 - val_loss: 0.4958 - val_accuracy: 0.7927
Epoch 4/5
22/22 [==============================] - 1s 27ms/step - loss: 0.3225 - accuracy: 0.8628 - val_loss: 0.5132 - val_accuracy: 0.7664
Epoch 5/5
22/22 [==============================] - 1s 49ms/step - loss: 0.2642 - accuracy: 0.8993 - val_loss: 0.5388 - val_accuracy: 0.7612


In [102]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.03548534],
       [0.6089234 ],
       [0.9844662 ],
       [0.45757076],
       [0.6633037 ],
       [0.52044976],
       [0.97809064],
       [0.92444426],
       [0.9765198 ],
       [0.19141018]], dtype=float32)

In [103]:
# Turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [104]:
# Evaluate model 7 predictions
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 76.11548556430446,
 'precision': 0.7608318753855237,
 'recall': 0.7611548556430446,
 'f1': 0.7609090208165951}